# Modelo de aprendizaje supervisado para la predicción del precio de alojamientos en Airbnb a partir de características de oferta y reputación del anfitrión

**1. Extracción de datos**

### Bloque 1
**Objetivo funcional:** Carga los datos finales de entrenamiento desde la capa Gold..
**Entradas/Salidas:** airbnb.gold.price_features

---


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("AirbnbPriceModel").getOrCreate()

df = spark.sql("SELECT * FROM airbnb.gold.price_features")


**2. Limpieza y selección**

### Bloque 2
**Objetivo funcional:** Carga los datos finales de entrenamiento desde la capa Gold..

---


In [0]:
from pyspark.sql.functions import col

df_clean = (
    df
    .filter(col("price").isNotNull())
    .filter(col("price") > 0)
    .dropna(subset=["accommodates", "bedrooms", "bathrooms"])
)


**3. Preparación de features**

### Bloque 3
**Objetivo funcional:** Prepara y entrena el modelo predictivo usando los datos de la capa Gold..

---


In [0]:
from pyspark.sql.functions import col

# convertir a string solo la columna conflictiva
df_clean = df_clean.withColumn("host_is_superhost", col("host_is_superhost").cast("string"))


from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml import Pipeline

cat_cols = ["country", "city", "neighborhood", "host_is_superhost"]
num_cols = [
    "accommodates", "bedrooms", "bathrooms", "beds",
    "review_scores_rating", "number_of_reviews", "host_total_listings_count",
    "host_response_rate", "amenities_count", "latitude", "longitude",
    "days_since_first_review", "days_since_last_review", "reviews_per_month_der"
]
bin_cols = [c for c in df.columns if c.startswith("has_") or c.startswith("is_")]

indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx") for c in cat_cols]
assembler = VectorAssembler(
    inputCols=num_cols + [f"{c}_idx" for c in cat_cols] + bin_cols,
    outputCol="features"
)

pipeline = Pipeline(stages=indexers + [assembler])
data = pipeline.fit(df_clean).transform(df_clean)



**4. División train/test**

### Bloque 4
**Objetivo funcional:** Paso funcional dentro del proceso de entrenamiento o evaluación del modelo..

---


In [0]:
train, test = data.randomSplit([0.8, 0.2], seed=42)


**5. Entrenamiento (modelo base: Random Forest Regressor)**

### Bloque 5
**Objetivo funcional:** Carga los datos finales de entrenamiento desde la capa Gold..

---


In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType

# 1  Asegurar double (ya lo tienes, pero lo dejamos robusto)
for c in assembler.getInputCols():
    data = data.withColumn(c, col(c).cast(DoubleType()))

# 2  Reemplazar NaN por 0 (solo para columnas numéricas)
data = data.na.fill(0, subset=assembler.getInputCols())

# 3  Si existe columna 'features' anterior, eliminarla
if 'features' in data.columns:
    data = data.drop('features')

# 4  Ensamblar nuevamente
data = assembler.transform(data)


### Bloque 6
**Objetivo funcional:** Prepara y entrena el modelo predictivo usando los datos de la capa Gold..

---


In [0]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

train, test = data.randomSplit([0.8, 0.2], seed=42)

rf = RandomForestRegressor(featuresCol="features", labelCol="price", numTrees=100)
model = rf.fit(train)

preds = model.transform(test)
evaluator = RegressionEvaluator(labelCol="price", predictionCol="prediction")

rmse = evaluator.evaluate(preds, {evaluator.metricName: "rmse"})
r2 = evaluator.evaluate(preds, {evaluator.metricName: "r2"})
print(f"RMSE: {rmse:.2f}, R²: {r2:.2f}")


**6. Interpretación de resultados**

- RMSE mide el error promedio en euros o USD por noche.
- R² indica cuánta variabilidad del precio explica el modelo.
- Se puede analizar la importancia de las variables:

### Bloque 7
**Objetivo funcional:** Paso funcional dentro del proceso de entrenamiento o evaluación del modelo..

---


In [0]:
import pandas as pd

importance = pd.DataFrame({
    "feature": assembler.getInputCols(),
    "importance": model.featureImportances.toArray()
}).sort_values("importance", ascending=False)

display(importance)


**7. Exportar el dataset - gold**

### Bloque 8
**Objetivo funcional:** Paso funcional dentro del proceso de entrenamiento o evaluación del modelo..

---


In [0]:
# 1 Prepara tu dataset limpio
cols = ["price"] + assembler.getInputCols()
df_final = data.select(*cols).na.fill(0)

# 2 Muestra los datos en la interfaz
display(df_final)
